In [ ]:
# Stažení textového obsahu
# knihovna requests
# !pip install requests (! = spusť v příkazové řádce pip install requests)

# Zajímavosti:
# - pokud budeme vícekrát importovat knihovnu, Python z toho udělá jeden import
# - import můžeme psát i do metod - kdybychom metodu pak někam kopírovali, je jasné, že ke svému chodu potřebuje tuto knihovnu

In [7]:
import requests

In [30]:
def url_to_textfile(url: str, file_name: str) -> None:
    """
    Get text from URL and save it to the file.
    """
    response = requests.get(url)
    content_type = response.headers["content-type"]
    if not content_type.startswith("text/"):
        raise Exception(f"Invalid content type {content_type}")
    if not response.status_code == 200:
        raise Exception(f"There was an error fetching website. Url: {url}")
    with open(file=file_name, mode="wt", encoding="utf-8") as file:
        file.write(response.text)

In [25]:
url_to_textfile(url="https://1url.cz/@jf_Svejk", file_name="svejk.txt")

In [ ]:
# Základní operace se souborem

In [40]:
with open(file="svejk.txt", mode="rt", encoding="utf-8") as file:
    characters_count = len(file.read())
    print(f"{characters_count} znaků")

1225440 znaků


In [39]:
# Paměťové náročné
# Načítáme celý soubor, a pak počítáme počet bytů.
with open(file="svejk.txt", mode="rb") as file:
    bytes_count = len(file.read())
    print(f"{bytes_count} bytů")

1380884 bytů


In [42]:
# Méně paměťově náročné, ale může být pomalé
# Načítáme soubor postupně a postupně počítáme počet bytů.
with open(file="svejk.txt", mode="rb") as file:
    bytes_count = 0
    while file.read(1):
        bytes_count += 1
    print(f"{bytes_count} bytů")

1380884 bytů


In [44]:
# Toto řešení je jak paměťově nenáročné, ale i rychlé.
# Přeneseme se na konec souboru, a to vrátí aktuální pozici - počet bytů.
import io

with open(file="svejk.txt", mode="rb") as file:
    # Přesunutí na konec
    position = file.seek(0, io.SEEK_END)
    print(f"{position} bytů")

1380884 bytů


In [46]:
# Zjištění velikosti souboru bez jeho otevření.
# Toto by mělo být nejrychlejší.
# Nepracuje s děravými soubory (Unix).
from pathlib import Path

print(f"{Path("svejk.txt").stat().st_size} bytů")

1380884 bytů


In [56]:
# max(file) = vrátí řádek, který začíná na takový unicode znak, který má největší hodnotu.
with open(file="svejk.txt", mode="rt", encoding="utf-8") as file:
    max_line = max(file)
    print(max_line)

…který v zájmu svém vodí Vás, milostivá, s sebou na divadelní představení odpovídající jedině jeho vkusu. Mám rád upřímnost, nevtírám se nijak ve Váš soukromý život, a přál bych si pohovořit s Vámi soukromě o čistém umění…



In [57]:
# Trojtečka
max_line[0]

'…'

In [58]:
# Vrátí Unicode kód
ord(max_line[0])

8230

In [59]:
# Nejdelší odstavec (vnímán jako řádek) ve Švejkovy.
# Nyní můžeme zpracovávat text po řádcích, protože nyní víme, že žádný řádek nebude moc dlouhý (delší než tento).
with open(file="svejk.txt", mode="rt", encoding="utf-8") as file:
    max_line = max(file, key=len)
    print(max_line)

„Není ten Chodounský, co má soukromej detektivní ústav s tím vokem jako trojice boží, váš příbuznej?“ otázal se nevinně Švejk. „Já mám moc rád soukromý detektivy. Já jsem taky jednou sloužil před léty na vojně s jedním soukromým detektivem, s nějakým Stendlerem. Ten měl tak šišatou hlavu, že mu náš feldvébl vždycky říkal, že už viděl moc šišatejch hlav vojenskejch za dvanáct let, ale takovou šišku že si ani v duchu nepředstavoval. ‚Poslouchají, Stendlere,‘ říkal mu vždycky, ‚kdyby nebyly letos manévry, tak by se vaše šišatá hlava ani na vojnu nehodila, ale takhle se bude alespoň podle vaší šišky artilérie nastřelovat, když přijdem do krajiny, kde nebude žádnej lepší punkt k orientaci.‘ Ten vod něho zkusil. Někdy, při marši, poslal ho na pět set kroků napřed a pak poručil: ‚Direktion šišatá hlava.‘ Von měl vůbec ten pan Stendler, i jako soukromej detektiv, náramnou smůlu. Kolikrát nám v kantýně o tom vyprávěl, jaký měl často trápení. Von dostával takový úlohy, jako kupříkladu vypátrat, 

In [100]:
# Regular Expression (regulární výraz).
# Jazyk, kterým lze popsat strukturu řetězců.
# \s = libovolný znak, který je mezerový/bílý (space character,white space)
# \s+ = libovolná posloupnost bílých znaků jdoucích za sebou
# [] = nebo, tedy [\s,.,;] = posloupnost bílých znaků, tečka nebo středník
# \W+ = pouze písmena, nic jiného
from collections import Counter
import re


def text_to_words(text: str) -> list[str]:
    return re.split(r"\W+", text)


def numbers_of_words(text: str) -> int:
    return len(text_to_words(text))


with open(file="svejk.txt", mode="rt", encoding="utf-8") as file:
    text = file.read()
    words = text_to_words(text)
    sentences = re.split(r"[.?!„“”]\s*", text)
    words_dict = Counter([word.lower() for word in words])
    print(f"Počet slov: {len(words)}")
    print(f"Nejdelší slovo: {max(words, key=len)}")
    print(f"Nejdelší věta: {max(sentences, key=len)}")
    print(f"Věta s nejvíce slovy: {max(sentences, key=numbers_of_words)}")
    print(
        f"Za jakou dobu přečteme Švejka (při rychlosti 200 slov/minutu): {len(words) / (200 * 60)} hodin"
    )
    print(
        f"Jaké slovo se vyskytuje ve Švejkovy nejčastěji: {Counter.most_common(words_dict)[0]}"
    )
    print(
        f"Všechny jednoslovné věty ve Švejkovy: {[sentence for sentence in sentences if numbers_of_words(sentence) == 1 and len(sentence) > 0]}"
    )
    print(f"Jaké písmeno se ve slovech vyskytuje nejčastěji (bez ohledu na pozici): {Counter("".join([word.lower() for word in words])).most_common()[0]}")
    print(f"Jaké písmeno se na začátku slov vyskytuje nejčastěji: {Counter([word[0] for word in words if word]).most_common()[0]}")

Počet slov: 200148
Nejdelší slovo: offiziersmenageküchenabteilung
Nejdelší věta: Převládali vojáci od různých regimentů, formací a nejrůznější národnosti, které vichřice válečná zavála do táborských lazaretů, kteří nyní odjížděli znova do pole pro nová zranění, zmrzačení, bolesti a jeli si vysloužit nad svými hroby prostý dřevěný kříž, na kterém ještě po letech na smutných pláních východní Haliče bude se ve větru a dešti třepetat vybledlá rakouská vojenská čepice se zrezavělým frantíkem, na které čas od času si usedne smutný zestárlý krkavec, vzpomínající na tučné hody před lety, kdy býval tu pro něho nekonečný prostřený stůl lidských chutných mrtvol a koňských zdechlin, kdy právě pod takovou čepicí, na které sedí, bylo to nejchutnější sousto – lidské oči
Věta s nejvíce slovy: Mladší sestra že se utopila, starší že se vrhla pod vlak, bratr že skočil z železničního mostu na Vyšehradě, dědeček že zavraždil svou ženu a polil se petrolejem a zapálil se, druhá babička že se toulala s cikány